In [1]:
from clickhouse_driver import Client

In [ ]:
database = 'metro'
table = 'station'
# Подключение к ClickHouse
client = Client(host='localhost', database=database)

# Создание базы данных
client.execute('CREATE DATABASE IF NOT EXISTS metro')

# Создание таблицы
client.execute('''
CREATE TABLE IF NOT EXISTS metro.station (
    C_A String,
    Unit String,
    SCP String,
    Station String,
    Date Date,
    Time String,
    Description String,
    Entries Int64,
    Exits Int64,
    datetime DateTime,
    turnstile String
) ENGINE = MergeTree()
ORDER BY (Station, datetime)
''')



query = f'''
SELECT count(*) 
FROM system.tables 
WHERE database = '{database}' AND name = '{table}'
'''

# Выполнение запроса
table_exists = client.execute(query)[0][0]

# Вывод результата
if table_exists:
    print(f"Таблица {table} существует в базе данных {database}.")
else:
    print(f"Таблица {table} не существует в базе данных {database}.")

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import seaborn as sns
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import mlab
from matplotlib.ticker import FuncFormatter
matplotlib.style.use('ggplot')
import sqlite3 
import warnings
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None

mta = pd.read_csv('Turnstile_Usage_Data__2022.csv', sep = ',',header = None)

# Присвоение имен столбцов
mta.columns = ['C/A', 'Unit', 'SCP', 'Station', 'Line Name', 'Division', 'Date', 'Time', 'Description', 'Entries', 'Exits']

# mta.columns =['C/A','unit','SCP','Station','date','time','desc','entries','exits']

# Оставляем только нужные столбцы
mta = mta[['C/A', 'Unit', 'SCP', 'Station', 'Date', 'Time', 'Description', 'Entries', 'Exits']]
mta = mta.drop(index=0)
print(mta.head())

In [ ]:
mta['datetime'] = pd.to_datetime(mta.Date + ' ' + mta.Time, format='%m/%d/%Y  %H:%M:%S')
mta['turnstile'] = mta['C/A'] + '-' + mta['Unit'] + '-' + mta['SCP']

mta = mta[(mta.datetime >= '01-01-18 00:00:00') & 
          (mta.datetime <'07-01-26 00:00:00')]

print(mta.head())

In [ ]:
print(mta.head())

# Преобразование строк в даты
mta['Date'] = pd.to_datetime(mta['Date'], infer_datetime_format=True, dayfirst=False).dt.date
mta['datetime'] = pd.to_datetime(mta['datetime'], infer_datetime_format=True, dayfirst=False)

# Преобразование строк в целые числа
mta['Entries'] = pd.to_numeric(mta['Entries'], errors='coerce').fillna(0).astype(int)
mta['Exits'] = pd.to_numeric(mta['Exits'], errors='coerce').fillna(0).astype(int)

# Преобразование данных в список кортежей
data = list(mta.itertuples(index=False, name=None))

# Вставка данных в таблицу с обработкой исключений
try:
    client.execute('''
    INSERT INTO metro.station (C_A,
    Unit ,
    SCP ,
    Station ,
    Date ,
    Time ,
    Description ,
    Entries ,
    Exits ,
    datetime ,
    turnstile ) VALUES
    ''', data)
except ValueError as ve:
    print(f"ValueError occurred: {ve}")
except TypeError as te:
    print(f"TypeError occurred: {te}")
# except client.Error as ce:
#     print(f"ClickHouse Error occurred: {ce}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
# Рассчитаем корреляционную матрицу
corr = mta[['Entries', 'Exits']].corr()

# Построим тепловую карту
plt.figure(figsize=(8, 6))
sns.heatmap(corr, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix')
plt.show()

In [ ]:
# Описание каждого атрибута
description = mta.describe(include='all')
description

In [ ]:
# Информация о количестве пустых значений для каждого атрибута
missing_values = mta.isnull().sum()
missing_values

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import shapiro, normaltest, kstest

# Функция для графического и статистического анализа плотности распределения
def plot_density(df, column):
    sns.histplot(df[column].dropna(), kde=True)
    plt.title(f'Density plot for {column}')
    plt.show()
    sns.boxplot(x=df[column].dropna())
    plt.title(f'Box plot for {column}')
    plt.show()
    print(df[column].describe())

In [ ]:
# Функция для графического и статистического анализа нормальности распределения данных
def plot_normality(df, column):
    # Q-Q plot
    from scipy.stats import probplot
    probplot(df[column].dropna(), dist="norm", plot=plt)
    plt.title(f'Q-Q plot for {column}')
    plt.show()
    
    # Шапиро-Уилка тест
    stat, p = shapiro(df[column].dropna())
    print(f'Shapiro-Wilk Test for {column}: Statistics={stat}, p-value={p}')
    
    # Д'Агостино тест
    stat, p = normaltest(df[column].dropna())
    print(f'D\'Agostino Test for {column}: Statistics={stat}, p-value={p}')
    
    # Критерий Колмогорова-Смирнова
    stat, p = kstest(df[column].dropna(), 'norm')
    print(f'Kolmogorov-Smirnov Test for {column}: Statistics={stat}, p-value={p}')
    
    if p > 0.05:
        print(f'{column} seems to follow a normal distribution (fail to reject H0)')
    else:
        print(f'{column} does not follow a normal distribution (reject H0)')

In [ ]:
# Применение функций к каждому числовому атрибуту
numeric_columns = mta.select_dtypes(include=[np.number]).columns

for column in numeric_columns:
    print(f'Attribute: {column}')
    plot_density(mta, column)
    plot_normality(mta, column)

In [ ]:
# Функция для определения общей загруженности одной станции
def total_traffic(data):
    total_entries = data['Entries'].iloc[-1] - data['Entries'].iloc[0]
    total_exits = data['Exits'].iloc[-1] - data['Exits'].iloc[0]
    total_traffic = total_entries + total_exits
    return total_traffic

In [ ]:
# Функция для расчета средней пропускной способности всех станций
def calculate_average_capacity(df):
    # Группировка данных по станциям и турникетам
    grouped = df.groupby(['Station', 'turnstile']).apply(total_traffic).reset_index(name='total_traffic')
    # Расчет средней пропускной способности
    avg_capacity = grouped['total_traffic'].mean()
    return avg_capacity

# Функция для расчета пропускной способности в процентах для одной станции
def capacity_percentage(data, avg_capacity):
    total_traffic_station = total_traffic(data)
    capacity_percent = (total_traffic_station / avg_capacity) * 100
    return capacity_percent

# Расчет средней пропускной способности
avg_capacity = calculate_average_capacity(data_other_stations)

# Расчет пропускной способности в процентах для одной станции
percentage = capacity_percentage(data, avg_capacity)

avg_capacity, percentage

In [ ]:
def entry_exit_statistics(data):
    total_entries = data['Entries'].iloc[-1] - data['Entries'].iloc[0]
    total_exits = data['Exits'].iloc[-1] - data['Exits'].iloc[0]
    return total_entries, total_exits

# Пример использования
entry_exit_statistics(data)

In [ ]:
def travel_intervals(data):
    intervals = data['datetime'].diff().dropna()
    return intervals

# Пример использования
travel_intervals(data)

In [ ]:
def travel_duration(data):
    intervals = travel_intervals(data)
    duration = intervals.dt.total_seconds() / 60  # Продолжительность в минутах
    return duration

# Пример использования
travel_duration(data)